<a href="https://colab.research.google.com/github/christymanthara/AgenticRag-with-Cohere/blob/main/AgenticRAGCohere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
!pip install -qU langchain langchain_openai langgraph arxiv duckduckgo-search
!pip install -qU faiss-cpu pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 3.0 MB/s eta 0:00:00


In [86]:
pip install langchain-cohere

In [87]:
from langchain_cohere import ChatCohere
llm = ChatCohere(cohere_api_key="ckHImzP6pm5GmqHOer9IAuAJhXPP05coM729kPac")

In [88]:
from google.colab import userdata
from uuid import uuid4
import os

In [89]:
!pip install langchain_community


In [90]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import ArxivLoader
from langchain_community.vectorstores import FAISS
from langchain_cohere.embeddings import CohereEmbeddings

# Load the document pertaining to a particular topic
docs = ArxivLoader(query="Retrieval Augmented Generation", load_max_docs=5).load()

# Split the dpocument into smaller chunks
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=350, chunk_overlap=50
)

chunked_documents = text_splitter.split_documents(docs)
#
# Instantiate the Embedding Model
embeddings = CohereEmbeddings(model="embed-english-light-v3.0",cohere_api_key="ckHImzP6pm5GmqHOer9IAuAJhXPP05coM729kPac")
# Create Index- Load document chunks into the vectorstore
faiss_vectorstore = FAISS.from_documents(
    documents=chunked_documents,
    embedding=embeddings,
)
# Create a retriver
retriever = faiss_vectorstore.as_retriever()

##Generate RAG prompt

In [91]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Use the following context to answer the user's query. If you cannot answer the question, please respond with 'I don't know'.

Question:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)


In [92]:
from langchain_cohere import ChatCohere

##instantiating the large language model

In [93]:
from langchain_core.messages import HumanMessage

chat_model = ChatCohere(cohere_api_key="ckHImzP6pm5GmqHOer9IAuAJhXPP05coM729kPac")

In [94]:
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

retrieval_augmented_generation_chain = (
       {"context": itemgetter("question")
    | retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)
#
retrieval_augmented_generation_chain

{
  context: RunnableLambda(itemgetter('question'))
           | VectorStoreRetriever(tags=['FAISS', 'CohereEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7e77605347f0>),
  question: RunnableLambda(itemgetter('question'))
}
| RunnableAssign(mapper={
    context: RunnableLambda(itemgetter('context'))
  })
| {
    response: ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="Use the following context to answer the user's query. If you cannot answer the question, please respond with 'I don't know'.\n\nQuestion:\n{question}\n\nContext:\n{context}\n"))])
              | ChatCohere(client=<cohere.client.Client object at 0x7e775512cbb0>, async_client=<cohere.client.AsyncClient object at 0x7e775512d000>, cohere_api_key=SecretStr('**********')),
    context: RunnableLambda(itemgetter('context'))
  }

In [103]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [104]:
await retrieval_augmented_generation_chain.ainvoke({"question" : "What is Retrieval Augmented Generation?"})

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/langchain_core/callbacks/manager.py", line 1997, in _configure
    handler = LangChainTracer(
  File "/usr/local/lib/python3.10/dist-packages/langchain_core/tracers/langchain.py", line 91, in __init__
    self.client = client or get_client()
  File "/usr/local/lib/python3.10/dist-packages/langchain_core/tracers/langchain.py", line 54, in get_client
    _CLIENT = Client()
  File "/usr/local/lib/python3.10/dist-packages/langsmith/client.py", line 534, in __init__
    _validate_api_key_if_hosted(self.api_url, self.api_key)
  File "/usr/local/lib/python3.10/dist-packages/langsmith/client.py", line 323, in _validate_api_key_if_hosted
    raise ls_utils.LangSmithUserError(
langsmith.utils.LangSmithUserError: API key must be provided when using hosted LangSmith API

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/pyth

{'response': AIMessage(content='Retrieval Augmented Generation (RAG) is a technique used to improve the performance of Large Language Models (LLMs). It involves retrieving relevant tools or information for a given task and providing them to the LLM to enhance its generation capabilities. RAG addresses the limitation of LLMs by providing access to external knowledge sources, improving their ability to solve new tasks with just a few examples.', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '1724a539-aab7-4417-9aeb-ca6e7eea4487', 'token_count': {'input_tokens': 2644, 'output_tokens': 77}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '1724a539-aab7-4417-9aeb-ca6e7eea4487', 'token_count': {'input_tokens': 2644, 'output_tokens': 77}}, id='run-4166cf7b-f3d9-4ace-86f9-21b2f28c997b-0'),


In [115]:
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langgraph.prebuilt import ToolExecutor
tool_belt = [
    DuckDuckGoSearchRun(),
    ArxivQueryRun()
]

tool_executor = ToolExecutor(tool_belt)

In [106]:
%pip install --upgrade --quiet  arxiv

In [107]:
!pip install langchainhub

In [108]:
%pip install --upgrade --quiet  duckduckgo-search

In [134]:
import cohere
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain_cohere.chat_models import ChatCohere
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.prompts import ChatPromptTemplate



llm = ChatCohere(model="command-r-plus", temperature=0.3)
# tools = load_tools(
#     ["duckduckgo-search"],["arxiv"]
# )
# prompt = hub.pull("hwchase17/react")

duckSearch = DuckDuckGoSearchRun()
duckSearch.name = "search_engine"
duckSearch.description = "returns a string as the answer";

from langchain_core.pydantic_v1 import BaseModel, Field
class DuckSearchInput(BaseModel):
   query: str = Field(description="Query to search the internet with")
duckSearch.args_schema = DuckSearchInput


tools = [duckSearch]






In [135]:
preamble = """
You are an expert who answers the user's question with the most relevant datasource.
You are equipped with an internet search tool and a special vectorstore of information
about how to write good essays.
"""

In [136]:
prompt = ChatPromptTemplate.from_template("{input}")
agent = create_cohere_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tool_belt, verbose=True)

In [137]:
agent_executor.invoke({
   "input": "I want to write an essay about the Roman Empire. Any tips for writing an essay? Any fun facts?",
   "preamble": preamble,
})

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/langchain_core/callbacks/manager.py", line 1997, in _configure
    handler = LangChainTracer(
  File "/usr/local/lib/python3.10/dist-packages/langchain_core/tracers/langchain.py", line 91, in __init__
    self.client = client or get_client()
  File "/usr/local/lib/python3.10/dist-packages/langchain_core/tracers/langchain.py", line 54, in get_client
    _CLIENT = Client()
  File "/usr/local/lib/python3.10/dist-packages/langsmith/client.py", line 534, in __init__
    _validate_api_key_if_hosted(self.api_url, self.api_key)
  File "/usr/local/lib/python3.10/dist-packages/langsmith/client.py", line 323, in _validate_api_key_if_hosted
    raise ls_utils.LangSmithUserError(
langsmith.utils.LangSmithUserError: API key must be provided when using hosted LangSmith API

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/pyth



> Entering new AgentExecutor chain...


--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/langchain_core/callbacks/manager.py", line 1997, in _configure
    handler = LangChainTracer(
  File "/usr/local/lib/python3.10/dist-packages/langchain_core/tracers/langchain.py", line 91, in __init__
    self.client = client or get_client()
  File "/usr/local/lib/python3.10/dist-packages/langchain_core/tracers/langchain.py", line 54, in get_client
    _CLIENT = Client()
  File "/usr/local/lib/python3.10/dist-packages/langsmith/client.py", line 534, in __init__
    _validate_api_key_if_hosted(self.api_url, self.api_key)
  File "/usr/local/lib/python3.10/dist-packages/langsmith/client.py", line 323, in _validate_api_key_if_hosted
    raise ls_utils.LangSmithUserError(
langsmith.utils.LangSmithUserError: API key must be provided when using hosted LangSmith API

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/pyth


I will search for tips on how to write an essay and fun facts about the Roman Empire.
{'tool_name': 'search_engine', 'parameters': {'query': 'tips for writing an essay'}}
search_engine is not a valid tool, try one of [duckduckgo_search, arxiv].
{'tool_name': 'search_engine', 'parameters': {'query': 'fun facts about the Roman Empire'}}
search_engine is not a valid tool, try one of [duckduckgo_search, arxiv].

  File "/usr/lib/python3.10/threading.py", line 973, in _bootstrap
    self._bootstrap_inner()
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/concurrent/futures/thread.py", line 83, in _worker
    work_item.run()
  File "/usr/lib/python3.10/concurrent/futures/thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
  File "/usr/local/lib/python3.10/dist-packages/langchain_core/runnables/base.py", line 2731, in transform
    yield from self._transform_stream_with_config(
  File "/usr/local/lib/python3.10/dist-packages/langchain_core/runnables/base.py", line 1747, in _transform_stream_with_config
    chunk: Output = context.run(next, iterator)  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/langchain_core/runnables/base.py", line 2695, in _transform
    for output in final_pipelin

...

0:52 Rome Was a Republic and an Empire. 1:43 Julius Caesar was not the first Roman Emperor. 2:24 Conquest through Inclusion. 3:12 Government Modeled on Family. 3:50 Mental Males. 4:35 Fast Food: A Roman Invention. 5:30 Loved Rotten Fish Paste. 6:02 Invented the Apartment & Return Policy. 6:49 Rome Was Once Three Empires. The Roman Empire was one of the most influential and significant civilizations in world history. At its peak, the empire stretched from North Africa to Britain, reigning over 60 million people. From its legendary beginnings and remarkable achievements to its eventual decline and fall, the Roman Empire is a fascinating topic full of little-known facts and intriguing trivia. In 476 AD, the Germanic barbarian Odoacer captured the city of Rome and forced the last Emperor of Rome, Romulus August, to give up his crown. Odoacer then occupied Rome and declared himself the King of Italy, marking the death of the Western Roman Empire, which had stood for over 1,000 years. After 

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/langchain_core/callbacks/manager.py", line 1997, in _configure
    handler = LangChainTracer(
  File "/usr/local/lib/python3.10/dist-packages/langchain_core/tracers/langchain.py", line 91, in __init__
    self.client = client or get_client()
  File "/usr/local/lib/python3.10/dist-packages/langchain_core/tracers/langchain.py", line 54, in get_client
    _CLIENT = Client()
  File "/usr/local/lib/python3.10/dist-packages/langsmith/client.py", line 534, in __init__
    _validate_api_key_if_hosted(self.api_url, self.api_key)
  File "/usr/local/lib/python3.10/dist-packages/langsmith/client.py", line 323, in _validate_api_key_if_hosted
    raise ls_utils.LangSmithUserError(
langsmith.utils.LangSmithUserError: API key must be provided when using hosted LangSmith API

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/pyth

Relevant Documents: 2,3
Cited Documents: 2,3
Answer: Here are some tips for writing an essay:
- Pick your topic, gather information, and make a plan.
- Start with your main idea, add details to support it, and wrap it up with a conclusion.
- Check your work for writing mistakes and make sure everything flows smoothly.
- Create a word bank.

Here are some fun facts about the Roman Empire:
- The Roman Empire was once three empires.
- At its peak, the empire stretched from North Africa to Britain, reigning over 60 million people.
- The Romans invented fast food.
- They also invented apartments and return policies.
- The Roman Empire fell in 476 AD when the Germanic barbarian Odoacer captured the city of Rome and forced the last Emperor of Rome, Romulus August, to give up his crown.
Grounded answer: Here are some tips for writing an essay:
- <co: 2>Pick your topic, gather information, and make a plan</co: 2>.
- <co: 2>Start with your main idea, add details to support it, and wrap it up wit

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/langchain_core/callbacks/manager.py", line 1997, in _configure
    handler = LangChainTracer(
  File "/usr/local/lib/python3.10/dist-packages/langchain_core/tracers/langchain.py", line 91, in __init__
    self.client = client or get_client()
  File "/usr/local/lib/python3.10/dist-packages/langchain_core/tracers/langchain.py", line 54, in get_client
    _CLIENT = Client()
  File "/usr/local/lib/python3.10/dist-packages/langsmith/client.py", line 534, in __init__
    _validate_api_key_if_hosted(self.api_url, self.api_key)
  File "/usr/local/lib/python3.10/dist-packages/langsmith/client.py", line 323, in _validate_api_key_if_hosted
    raise ls_utils.LangSmithUserError(
langsmith.utils.LangSmithUserError: API key must be provided when using hosted LangSmith API

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/pyth

{'input': 'I want to write an essay about the Roman Empire. Any tips for writing an essay? Any fun facts?',
 'preamble': "\nYou are an expert who answers the user's question with the most relevant datasource.\nYou are equipped with an internet search tool and a special vectorstore of information\nabout how to write good essays.\n",
 'output': 'Here are some tips for writing an essay:\n- Pick your topic, gather information, and make a plan.\n- Start with your main idea, add details to support it, and wrap it up with a conclusion.\n- Check your work for writing mistakes and make sure everything flows smoothly.\n- Create a word bank.\n\nHere are some fun facts about the Roman Empire:\n- The Roman Empire was once three empires.\n- At its peak, the empire stretched from North Africa to Britain, reigning over 60 million people.\n- The Romans invented fast food.\n- They also invented apartments and return policies.\n- The Roman Empire fell in 476 AD when the Germanic barbarian Odoacer capture